In [1]:
# imports
import numpy as np
import cv2
from PIL import Image
from scipy.fftpack import dct
from scipy.fftpack import idct
from math import log10, sqrt

In [4]:
# Specify image files here

input_image = "/content/cover.jpg"
watermark_logo = "/content/iitbbs_logo.jpeg"
recovered_logo = '/content/recovered_watermark.png'
output_image = '/content/watermarked_image.png'


In [5]:
# Convert Image to grayscale and generate image array
def convert_image(image_name, size):   
  '''
  img = Image.open(image_name).resize((size, size), 1)  
  img = img.convert('L')  
  img.save(image_name)
  image_array = np.array(img.getdata(), dtype = float).reshape((size, size))              
  return image_array
  '''
  # Read original image and resize it to 1024x1024
  original = cv2.imread(image_name)
  original = cv2.resize(original, (size, size))
  # Convert image to float array and reshape it to (1024, 1024, 3)
  image_array = np.array(original, dtype=float).reshape((size, size, 3))

  return original,image_array


In [6]:
# Read watermark image and resize it to 128x128
def convert_watermark(watermark_path, size): 

    watermark_img = cv2.imread(watermark_path)
    watermark_img = cv2.resize(watermark_img, (size, size))
    # Convert watermark image to grayscale and convert it to float array
    watermark_img = cv2.cvtColor(watermark_img, cv2.COLOR_BGR2GRAY)
    image_array = np.array(watermark_img, dtype=float).reshape((size, size))

    return watermark_img, image_array

In [7]:
# Discrete Cosine Transform
def apply_dct(image_array): 
  size = image_array[0].__len__()  
  all_subdct = np.empty((size, size))  
  
  for i in range (0, size, 8):  
    for j in range (0, size, 8):          
      subpixels = image_array[i:i+8, j:j+8]         
      subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")         
      all_subdct[i:i+8, j:j+8] = subdct

  return all_subdct

In [8]:
# Embed watermark in given image
def embed_watermark(watermark_array, orig_image):   
  watermark_array_size = watermark_array[0].__len__()    
  watermark_flat = watermark_array.ravel()    
  ind = 0
  
  for x in range (0, orig_image.__len__(), 8):      
    for y in range (0, orig_image.__len__(), 8):        
      if ind < watermark_flat.__len__():           
        subdct = orig_image[x:x+8, y:y+8]             
        subdct[5][5] = watermark_flat[ind]              
        orig_image[x:x+8, y:y+8] = subdct            
        ind += 1 

  return orig_image

In [9]:
# Inverse DCT
def inverse_dct(all_subdct):   
  size = all_subdct[0].__len__()   
  all_subidct = np.empty((size, size))  

  for i in range (0, size, 8):   
      for j in range (0, size, 8):    
        subidct = idct(idct(all_subdct[i:i+8, j:j+8].T, norm="ortho").T, norm="ortho")      
        all_subidct[i:i+8, j:j+8] = subidct
        
  return all_subidct

In [10]:
# Recovered watermark
def recover_watermark(watermarked_image_array, output_logo):
  dct_watermarked_image_array = apply_dct(watermarked_image_array)  

  watermark_size = 128   
  subwatermarks = []

  for x in range (0, dct_watermarked_image_array.__len__(), 8):     
    for y in range (0, dct_watermarked_image_array.__len__(), 8):           
      coeff_slice = dct_watermarked_image_array[x:x+8, y:y+8]        
      subwatermarks.append(coeff_slice[5][5])
  
  watermark = np.array(subwatermarks).reshape(watermark_size, watermark_size)*255
  watermark =  np.uint8(watermark)

  img = Image.fromarray(watermark)
  img.save(output_logo)

In [11]:
# Calculate PSNR
def PSNR(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    if(mse == 0):  
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr

In [12]:
# Correlation coefficient
def correlation_coefficient(img1, img2):
    mean1 = np.mean(img1)
    mean2 = np.mean(img2)
    std1 = np.std(img1)
    std2 = np.std(img2)
    cov = np.mean((img1 - mean1) * (img2 - mean2))
    corr = cov / (std1 * std2)
    return corr

## Main code


In [13]:
#Converting the image
image, image_array = convert_image(input_image, 1024)

watermark, watermark_array = convert_watermark(watermark_logo, 128)


# Normalize watermark array to range [0, 1]
watermark_array = watermark_array / 255

# applying dct and embedding watermark on image
dct_array = apply_dct(image_array[:, :, 0])   

#embedding the watermark
dct_array = embed_watermark(watermark_array, dct_array)   

#getting final image
watermarked_image_array = inverse_dct(dct_array)


# Create a new image array with the watermarked blue channel
image_wm = np.zeros((1024, 1024, 3))
image_wm[:, :, 0] = watermarked_image_array
image_wm[:, :, 1] = image_array[:, :, 1]
image_wm[:, :, 2] = image_array[:, :, 2]

# Write the watermarked image to a file
cv2.imwrite(output_image, image_wm)
print("The watermark has been successfully embedded")

The watermark has been successfully embedded


In [14]:
original_image = cv2.imread(input_image)
watermarked_image = cv2.imread(output_image) 
psnr = PSNR(image, watermarked_image)
print("PSNR =", psnr)

PSNR = 57.34410646571743


In [15]:
# Recover watermark from image
recover_watermark(watermarked_image_array, recovered_logo)
print("Recovered logo =", recovered_logo)

Recovered logo = /content/recovered_watermark.png


In [16]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)

Correlation Coefficient = 0.9801455365336263


# Attacks

### Rotational Attack

In [17]:
img = cv2.imread(output_image)
img = cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2GRAY)
center = (512, 512)
rotation_matrix = cv2.getRotationMatrix2D(center, 90, 1)
tampered_image = cv2.warpAffine(img, rotation_matrix, (1024, 1024))
cv2.imwrite('rotational_attack.png', tampered_image)
print("Rotational attack done", 'rotational_attack.png')

Rotational attack done rotational_attack.png


In [18]:
# Recover watermark from image
recovered_logo = 'rotational_attack_recovered_watermark.png'
recover_watermark(tampered_image, recovered_logo)
print("Recovered logo =", recovered_logo)

Recovered logo = rotational_attack_recovered_watermark.png


In [19]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)

Correlation Coefficient = -0.007678179457582272


In [20]:
# Calculate PSNR between original logo and recovered logo

img1 = cv2.imread(watermark_logo)
img1 = cv2.resize(img1, (128, 128))
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

img2 = cv2.imread(recovered_logo)
img2 = cv2.resize(img2, (128, 128))
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

psnr = PSNR(img1, img2)
print("PSNR =", psnr)

PSNR = 27.889870170009374


### Average Filer

In [21]:
img = cv2.imread(output_image, 0)

m = img.shape[0]
n = img.shape[1]

filter = np.ones([3,3], dtype = int)
filter = filter/9

tampered_image = np.zeros([m,n])

for i in range(1, m-1):
  for j in range(1, n-1):
      temp = (img[i-1, j-1]*filter[0, 0])+(img[i-1, j]*filter[0, 1])+(img[i-1, j + 1]*filter[0, 2])+(img[i, j-1]*filter[1, 0])+ (img[i, j]*filter[1, 1])+(img[i, j + 1]*filter[1, 2])+(img[i + 1, j-1]*filter[2, 0])+(img[i + 1, j]*filter[2, 1])+(img[i + 1, j + 1]*filter[2, 2])         
      tampered_image[i,j] = temp

cv2.imwrite('average_filter_attack.png', tampered_image)
print("Average filter attack done", 'average_filter_attack.png')

Average filter attack done average_filter_attack.png


In [22]:
# Recover watermark from image
recovered_logo = 'average_filter_attack_recovered_watermark.png'
recover_watermark(tampered_image, recovered_logo)
print("Recovered logo =", recovered_logo)

Recovered logo = average_filter_attack_recovered_watermark.png


In [23]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)

Correlation Coefficient = -0.0289581949981876


In [24]:
# Calculate PSNR between original logo and recovered logo

img1 = cv2.imread(watermark_logo)
img1 = cv2.resize(img1, (128, 128))
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

img2 = cv2.imread(recovered_logo)
img2 = cv2.resize(img2, (128, 128))
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

psnr = PSNR(img1, img2)
print("PSNR =", psnr)

PSNR = 28.651303840251288


### Median Filter

In [25]:
img = cv2.imread(output_image, 0)
m, n = img.shape

tampered_image = np.zeros([m, n])
  
for i in range(1, m-1):
    for j in range(1, n-1):
        temp = [img[i-1, j-1],
              img[i-1, j],
              img[i-1, j + 1],
              img[i, j-1],
              img[i, j],
              img[i, j + 1],
              img[i + 1, j-1],
              img[i + 1, j],
              img[i + 1, j + 1]]
          
        temp = sorted(temp)
        tampered_image[i, j] = temp[4]
  
tampered_image = tampered_image.astype(np.uint8)
cv2.imwrite('median_filter_attack.png', tampered_image)
print("Median filter attack done", 'median_filter_attack.png')

Median filter attack done median_filter_attack.png


In [26]:
# Recover watermark from image
recovered_logo = 'median_filter_attack_recovered_watermark.png'
recover_watermark(tampered_image, recovered_logo)
print("Recovered logo =", recovered_logo)

Recovered logo = median_filter_attack_recovered_watermark.png


In [27]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)

Correlation Coefficient = -0.08458510209711666


### Cropping Attack

In [28]:
img = Image.open(output_image)
tampered_image = img.crop( (1, 0, 1022, 1023) )
tampered_image.save('cropping_attack.png')
print("Cropping attack done", 'cropping_attack.png')

Cropping attack done cropping_attack.png


In [29]:
# Recover watermark from image
recovered_logo = 'cropping_attack_recovered_watermark.png'
img = convert_image("cropping_attack.png", 1024)
recover_watermark(img, recovered_logo)
print("Recovered logo =", recovered_logo)

TypeError: ignored

In [ ]:
# Calculate Correlation coefficient between original logo and recovered logo
img1 = convert_image(watermark_logo, 128)
img2 = convert_image(recovered_logo, 128)
corr_coeff = correlation_coefficient(img1, img2)
print("Correlation Coefficient =", corr_coeff)